In [ ]:
import os
import open3d as o3d
from utils.imageProcessing import *
from utils.dicomFiles import *
from utils.plotting import *

In [ ]:
# load the DICOM files
dcm_path = './dicom-files/arq3/pt2'

files = []

for f in os.listdir(dcm_path):
    filename = os.path.join(dcm_path, f)
    if filename[-3:] == 'dcm':
        files.append(pydicom.dcmread(filename))

# skip files with no SliceLocation (eg scout views)

slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
        slices.append(f)
    else:
        skipcount = skipcount + 1

slices = list(map(getImgFromDicomFile, sorted(slices, key=lambda s: s.SliceLocation)))

In [ ]:
original = slices[85]
mask = identifyVentricle(original)
contour_image = drawContour(original.copy(), mask)
compareTwoImages(mask, contour_image)

In [ ]:
vertex = []
for Z in range(len(slices)):
    mask = identifyVentricle(slices[Z])
    edges =  cv2.Canny(mask, 85, 200)
    points = edges
    for X in range(len(points)):
        for Y in range(len(points[X])):
                if points[X][Y] == 255:
                    vertex.append((X,Y,5*Z))
vertex = np.array(vertex, dtype=np.float64)
x = vertex[:, 0]
y = vertex[:, 1]
z = vertex[:, 2]
len(z)

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(vertex)
o3d.visualization.draw_geometries([pcd])
